# 1 - Practical

In [24]:
import numpy as np
import math
from scipy.stats import norm

The objective of this part of the coursework is to implement three different ways of valuing put and call options.

Take two numbers, $X_1 = 120$ and $X_2$ calculated as follows. Take your student number, remove zeros and use the remaining digits to make the number closest to $100$. For example, if your student number is $100966240$, after dropping 0s you get $196624$ and can make the number $96$. Do not put your student number on your submission to preserve anonymity.

Consider options with strike prices $X_1$ and $X_2$ of $4$ types (European and American puts and calls) with the time to maturity equal to 6 months (1/2 of a year). 

In total this amounts to 8 different options.
Suppose the following holds:
- current stock price S0 is 100p;
- annual volatility is 20%;
- money can be invested at the annual interest rate (with continuous compounding) of 5% and the investment is virtually riskless; money can be borrowed at the same rate.

You should code up the Black–Scholes formula, binomial trees, and Monte Carlo algorithm, and find the following 20 numbers:
- the prices of the European call and put options computed from the Black–Scholes formula (4 numbers);
- the prices of the European and American call and put options computed using the binomial tree method (with approximately 1 day as the time step) (8 numbers);
- the prices of the European call and put options computed using the Monte Carlo method (with approximately 1 day as the time step) (4 numbers).

In the lectures several variations of each method were considered (e.g., different types of trees or simulations based on different SDEs). It is sufficient to choose one variant. Doing several variants will be considered an extra advantage, but it is not required.
- Hint 1: It is easier to measure time in years (fractions of a year), rather than in days.
- Hint 2: Different methods should return similar values for the same price.
For example, the prices of European options calculated using the Black-Scholes formula, binomial trees, and Monte-Carlo should be the same (or very close, usually within 0.1p).
- Hint 3: If you use Java, you will need to calculate the distribution function N of the Gaussian distribution ϕ(0, 1). You can use the method from the file CNDF.java provided on Moodle.

## Binomial Trees

In [25]:
def binomial_tree(S0: int, X: int, r: float, sigma: float, T: float, N: int, option: str, option_type: str):
    dt: float = T/N
    u: float = np.exp(sigma*np.sqrt(dt))
    d: float = 1/u
    p: float = (np.exp(r*dt) - d) / (u - d)

    stock_price: np.array = np.zeros((N+1, N+1))
    option_price: np.array = np.zeros((N+1, N+1))

    for i in range(N+1):
        for j in range(i+1):
            stock_price[j][i] = S0*(u**(i-j))*(d**j)

    if option_type == 'call' and option == 'european':
        option_price[:, N] = np.maximum(np.zeros(N+1), stock_price[:, N] - X)
    elif option_type == 'put' and option == 'european':
        option_price[:, N] = np.maximum(np.zeros(N+1), X - stock_price[:, N])
    elif option_type == 'call' and option == 'american':
        option_price[:, N] = np.maximum(np.zeros(N+1), stock_price[:, N] - X)
    elif option_type == 'put' and option == 'american':
        option_price[:, N] = np.maximum(np.zeros(N+1), X - stock_price[:, N])
    else:
        raise ValueError("Invalid option type")

    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_price[j][i] = (
                p*option_price[j][i+1] + (1-p)*option_price[j+1][i+1])/np.exp(r*dt)
            if option_type == 'call' and option == 'american':
                option_price[j][i] = np.maximum(
                    option_price[j][i], stock_price[j][i] - X)
            elif option_type == 'put' and option == 'american':
                option_price[j][i] = np.maximum(
                    option_price[j][i], X - stock_price[j][i])

    return option_price[0][0]


In [26]:
S0: int = 100
X1: int = 120
X2: int = 96  # This value is based on your student number as described in the question
r: float = 0.05
sigma: float = 0.2
T: float = 0.5
N: int = 365  # Approximate number of days in 6 months

european_call_X1 = binomial_tree(S0, X1, r, sigma, T, N, 'european', 'call')
european_put_X1 = binomial_tree(S0, X1, r, sigma, T, N, 'european', 'put')
american_call_X1 = binomial_tree(S0, X1, r, sigma, T, N, 'american' ,'call')
american_put_X1 = binomial_tree(S0, X1, r, sigma, T, N, 'american' , 'put')
european_call_X2 = binomial_tree(S0, X2, r, sigma, T, N, 'european', 'call')
european_put_X2 = binomial_tree(S0, X2, r, sigma, T, N, 'european', 'put')
american_call_X2 = binomial_tree(S0, X2, r, sigma, T, N, 'american' , 'call')
american_put_X2 = binomial_tree(S0, X2, r, sigma, T, N, 'american' , 'put')

print("European call option X1: ", european_call_X1)
print("European put option X1: ", european_put_X1)
print("American call option X1: ", american_call_X1)
print("American put option X1: ", american_put_X1)
print("European call option X2: ", european_call_X2)
print("European put option X2: ", european_put_X2)
print("American call option X2: ", american_call_X2)
print("American put option X2: ", american_put_X2)

European call option X1:  1.021482664849677
European put option X1:  18.058672108250214
American call option X1:  1.021482664849677
American put option X1:  20.0
European call option X2:  9.22387521633163
European put option X2:  2.8536267710520584
American call option X2:  9.22387521633163
American put option X2:  2.985307430987258


## Monte Carlo

In [27]:
def monte_carlo(S0, X, r, sigma, T, N, M, option_type):
    dt = T/N
    stock_price = np.zeros((M, N+1))
    stock_price[:, 0] = S0
    exercise = np.zeros((M, N+1))

    for i in range(1, N+1):
        z = np.random.normal(size=M)
        stock_price[:, i] = stock_price[:, i-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)

    if option_type == 'call':
        option_price = np.maximum(np.zeros(M), stock_price[:, N] - X)
    elif option_type == 'put':
        option_price = np.maximum(np.zeros(M), X - stock_price[:, N])
    elif option_type == 'american_call':
        for i in range(N-1, -1, -1):
            for j in range(M):
                if stock_price[j][i] - X > exercise[j][i+1]:
                    exercise[j][i] = stock_price[j][i] - X
                else:
                    exercise[j][i] = exercise[j][i+1]/np.exp(r*dt)
        option_price = np.maximum(np.zeros(M), exercise[:, 0])
    elif option_type == 'american_put':
        for i in range(N-1, -1, -1):
            for j in range(M):
                if X - stock_price[j][i] > exercise[j][i+1]:
                    exercise[j][i] = X - stock_price[j][i]
                else:
                    exercise[j][i] = exercise[j][i+1]/np.exp(r*dt)
        option_price = np.maximum(np.zeros(M), exercise[:, 0])
    else:
        raise ValueError("Invalid option type")

    option_price = np.exp(-r*T) * np.mean(option_price)

    return option_price


In [30]:
S0 = 100
X1 = 120
X2 = 96  # This value is based on your student number as described in the question
r = 0.05
sigma = 0.2
T = 0.5
N = 365  # Approximate number of days in 6 months
M = 1000  # Number of simulations

european_call_X1 = monte_carlo(S0, X1, r, sigma, T, N, M, 'call')
european_put_X1 = monte_carlo(S0, X1, r, sigma, T, N, M, 'put')
american_call_X1 = monte_carlo(S0, X1, r, sigma, T, N, M, 'american_call')
american_put_X1 = monte_carlo(S0, X1, r, sigma, T, N, M, 'american_put')
european_call_X2 = monte_carlo(S0, X2, r, sigma, T, N, M, 'call')
european_put_X2 = monte_carlo(S0, X2, r, sigma, T, N, M, 'put')
american_call_X2 = monte_carlo(S0, X2, r, sigma, T, N, M, 'american_call')
american_put_X2 = monte_carlo(S0, X2, r, sigma, T, N, M, 'american_put')

print("European call option X1: ", european_call_X1)
print("European put option X1: ", european_put_X1)
print("American call option X1: ", american_call_X1)
print("American put option X1: ", american_put_X1)
print("European call option X2: ", european_call_X2)
print("European put option X2: ", european_put_X2)
print("American call option X2: ", american_call_X2)
print("American put option X2: ", american_put_X2)

European call option X1:  0.9658327616801856
European put option X1:  17.995626345857467
American call option X1:  1.674776307178899
American put option X1:  28.33511744280988
European call option X2:  9.094276806893994
European put option X2:  2.798044569632484
American call option X2:  15.762927974075719
American put option X2:  5.9974955448639315


## Black-Scholes

In [31]:
def black_scholes(S0, X, r, sigma, T, option_type):
    d1 = (np.log(S0/X) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == 'call':
        option_price = S0*norm.cdf(d1) - X*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == 'put':
        option_price = X*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type")

    return option_price

In [32]:
S0 = 100
X1 = 120
X2 = 96  # This value is based on your student number as described in the question
r = 0.05
sigma = 0.2
T = 0.5

european_call_X1 = black_scholes(S0, X1, r, sigma, T, 'call')
european_put_X1 = black_scholes(S0, X1, r, sigma, T, 'put')
european_call_X2 = black_scholes(S0, X2, r, sigma, T, 'call')
european_put_X2 = black_scholes(S0, X2, r, sigma, T, 'put')

print("European call option X1: ", european_call_X1)
print("European put option X1: ", european_put_X1)
print("European call option X2: ", european_call_X2)
print("European put option X2: ", european_put_X2)

European call option X1:  1.0226152225558796
European put option X1:  18.05980466595578
European call option X2:  9.222233623917731
European put option X2:  2.8519851786376655
